# BIOS274, Class 6

# os module

In [1]:
# Operating system module
import os

In [2]:
# os.mkdir() - make a new directory
# os.remove() - remove a file
# os.chdir() - change directory, ie. move into another directory
# os.rmdir() - remove a directory, it must be empty before it can be removed
# os.getcwd() - get the current working directory
# os.listdir() - list all files and directories in the current directory

In [3]:
result = os.getcwd()
print(result)

/Users/cherry/Box Sync/BIOS274/BIOS274-2019/classes/class06


In [4]:
result = os.listdir()
print(result)

['class06-URL-functions-map.ipynb', '.ipynb_checkpoints']


In [6]:
dir_name = 'new_dir_123456'

result = os.mkdir(dir_name)
print(result)

FileExistsError: [Errno 17] File exists: 'new_dir_123456'

In [8]:
try:
    os.mkdir(dir_name)
except FileExistsError:
    print('Directory not created. Directory', dir_name, 'exists.')

Directory not created. Directory new_dir_123456 exists.


In [10]:
result = os.listdir()
print(result)

['new_dir_123456', 'class06-URL-functions-map.ipynb', '.ipynb_checkpoints']


In [12]:
result = os.chdir('new_dir_123456')
print(result)

FileNotFoundError: [Errno 2] No such file or directory: 'new_dir_123456'

In [13]:
try:
    os.chdir(dir_name)
except FileNotFoundError:
    print('No such file or directory')

No such file or directory


In [14]:
result = os.getcwd()
print(result)

/Users/cherry/Box Sync/BIOS274/BIOS274-2019/classes/class06/new_dir_123456


In [15]:
result = os.listdir()
print(result)

[]


In [16]:
print('Writing test file in new_dir_123456, testing.txt')
with open('testing.txt', 'w') as TEST:
    TEST.write('testing')

Writing test file in new_dir_123456, testing.txt


In [17]:
result = os.listdir()
print(result)

['testing.txt']


In [18]:
result = os.remove('testing.txt')
print(result)

None


In [19]:
result = os.chdir('..')
print(result)

None


In [20]:
result = os.rmdir('new_dir_123456')
print(result)

None


In [21]:
result = os.getcwd()
print(result)

/Users/cherry/Box Sync/BIOS274/BIOS274-2019/classes/class06


In [22]:
result = os.listdir()
print(result)

['class06-URL-functions-map.ipynb', '.ipynb_checkpoints']


# gzip & urllib

In [24]:
# Module for URL, interacting with the web
import urllib.request

# Module for reading and writing the compressed file format, gzip
import gzip

In [26]:
# GFF/GTF format

# ensEMBL GFF format: https://uswest.ensembl.org/info/website/upload/gff.html
SEQNAME = 0
SOURCE = 1
FEATURE = 2
START = 3
END = 4
SCORE = 5
STRAND = 6
FRAME = 7
ATTRIBUTE = 8

# FTP ensEMBL downloads site: https://uswest.ensembl.org/info/data/ftp/index.html

In [29]:
#GTF_URL = 'ftp://ftp.ensembl.org/pub/release-98/gtf/homo_sapiens/Homo_sapiens.GRCh38.98.chr.gtf.gz'

GTF_URL = 'https://web.stanford.edu/class/gene211/handouts/Homo_sapiens.GRCh38.chr21-truncated.gtf.gz'

In [30]:
# Another format for feature annotation is GFF3, slightly easier to parse
# ftp://ftp.ensembl.org/pub/release-98/gff3/homo_sapiens

GFF3_URL = 'https://web.stanford.edu/class/gene211/handouts/Homo_sapiens.GRCh38.98.chr21-partial.gff3.gz'

In [32]:
response = urllib.request.urlopen(GTF_URL)
genes = {}

line_count = 0

with gzip.open(response, 'rt') as inFile:
    for line in inFile:
        # skip file comment lines
        if line.startswith('#'):
            continue
        if line_count < 2:
            print(line.rstrip('\n'))
        line_count += 1
#

21	havana	gene	5011799	5017145	.	+	.	gene_id "ENSG00000279493"; gene_version "1"; gene_name "FP565260.4"; gene_source "havana"; gene_biotype "protein_coding";
21	havana	transcript	5011799	5017145	.	+	.	gene_id "ENSG00000279493"; gene_version "1"; transcript_id "ENST00000624081"; transcript_version "1"; gene_name "FP565260.4"; gene_source "havana"; gene_biotype "protein_coding"; transcript_name "FP565260.4-201"; transcript_source "havana"; transcript_biotype "protein_coding"; tag "cds_start_NF"; tag "mRNA_start_NF"; tag "basic"; transcript_support_level "5";


In [33]:
# Just to compare GTF and GFF3. The difference is in the ATTRIBUTE column (index 8)
response = urllib.request.urlopen(GFF3_URL)
genes = {}

line_count = 0

with gzip.open(response, 'rt') as inFile:
    for line in inFile:
        # skip file comment lines
        if line.startswith('#'):
            continue
        if line_count < 2:
            print(line.rstrip('\n'))
        line_count += 1

21	Ensembl	chromosome	1	46709983	.	.	.	ID=chromosome:21;Alias=CM000683.2,chr21,NC_000021.9
21	havana	gene	5011799	5017145	.	+	.	ID=gene:ENSG00000279493;Name=FP565260.4;biotype=protein_coding;description=novel protein%2C similar to DNA (cytosine-5-)-methyltransferase 3-like DNMT3L;gene_id=ENSG00000279493;logic_name=havana_homo_sapiens;version=1


In [35]:
response = urllib.request.urlopen(GTF_URL)
genes = {}

attribute_names = ['gene_id', 'gene_name', 'gene_biotype']

with gzip.open(response, 'rt') as inFile:
    for line in inFile:
        # skip file comment lines
        if line.startswith('#'):
            continue

        # split line into columns that are separated by TAB 
        column = line.rstrip('\n').split('\t')

        # manipulate column #8, attributes
        if column[FEATURE] == 'gene':
            # starting ATTRIBUTE string
            # gene_id "ENSG00000279493"; gene_version "1"; gene_name "FP565260.4";
            
            # remove semi-colon from right
            # gene_id "ENSG00000279493"; gene_version "1"; gene_name "FP565260.4"            
            column[ATTRIBUTE] = column[ATTRIBUTE].rstrip(';')
            
            # replace semi-colon space with just semi-colon, and delete double-quotes
            # then split on semi-colon
            # ['gene_id ENSG00000279493', 'gene_version 1', 'gene_name FP565260.4']
            attribute = column[ATTRIBUTE].replace('; ', ';').replace('"', '').split(';')

            for item in attribute:
                #print(item)
                name, value = item.split(' ')
                if name == 'gene_id':
                    gene_id = value
                if name == 'gene_name':
                    gene_name = value
                if name == 'gene_biotype':
                    gene_biotype = value
                    
            genes[gene_name] = {'gene_id': gene_id,
                                'chromsome': int(column[SEQNAME]),
                                'start': int(column[START]),
                                'end': int(column[END]),
                                'biotype': gene_biotype
                                }

            if gene_name == 'GATD3B':
                print(genes['GATD3B'])

{'gene_id': 'ENSG00000280071', 'chromsome': 21, 'start': 5079294, 'end': 5128413, 'biotype': 'protein_coding'}


In [37]:
print(genes)

{'FP565260.4': {'gene_id': 'ENSG00000279493', 'chromsome': 21, 'start': 5011799, 'end': 5017145, 'biotype': 'protein_coding'}, 'FP565260.3': {'gene_id': 'ENSG00000277117', 'chromsome': 21, 'start': 5022493, 'end': 5040666, 'biotype': 'protein_coding'}, 'FP565260.5': {'gene_id': 'ENSG00000279687', 'chromsome': 21, 'start': 5073458, 'end': 5087867, 'biotype': 'lncRNA'}, 'GATD3B': {'gene_id': 'ENSG00000280071', 'chromsome': 21, 'start': 5079294, 'end': 5128413, 'biotype': 'protein_coding'}, 'FP565260.2': {'gene_id': 'ENSG00000276612', 'chromsome': 21, 'start': 5116343, 'end': 5133805, 'biotype': 'protein_coding'}, 'FP565260.1': {'gene_id': 'ENSG00000275464', 'chromsome': 21, 'start': 5130871, 'end': 5154658, 'biotype': 'protein_coding'}, 'FP565260.6': {'gene_id': 'ENSG00000280433', 'chromsome': 21, 'start': 5155499, 'end': 5165472, 'biotype': 'protein_coding'}, 'AC079801.1': {'gene_id': 'ENSG00000279669', 'chromsome': 21, 'start': 5232668, 'end': 5243833, 'biotype': 'lncRNA'}, 'LINC01670'

In [38]:
for key in genes:
    print(key)

FP565260.4
FP565260.3
FP565260.5
GATD3B
FP565260.2
FP565260.1
FP565260.6
AC079801.1
LINC01670
CU633967.1
Y_RNA
FP236315.2
FP236315.3
FP236315.1
CU639417.1
CU639417.3
CU639417.2
LINC01669
CU639417.4
CU639417.5
SIK1B
CU633906.5
CU633906.1
CU633906.4
CU633906.3
CU633906.2
CBSL
U2AF1L5


In [39]:
for key in genes:
    if not key.startswith('FP') and not key.startswith('CU'):
        print(key)

GATD3B
AC079801.1
LINC01670
Y_RNA
LINC01669
SIK1B
CBSL
U2AF1L5


In [40]:
# Biotypes
#
# TEC (To be Experimentally Confirmed)
# Protein coding
# Pseudogene
# ncRNA

In [41]:
for key in genes:
    if genes[key] ['biotype'] != 'protein_coding':
        print(key, genes[key])

FP565260.5 {'gene_id': 'ENSG00000279687', 'chromsome': 21, 'start': 5073458, 'end': 5087867, 'biotype': 'lncRNA'}
AC079801.1 {'gene_id': 'ENSG00000279669', 'chromsome': 21, 'start': 5232668, 'end': 5243833, 'biotype': 'lncRNA'}
LINC01670 {'gene_id': 'ENSG00000279094', 'chromsome': 21, 'start': 5499151, 'end': 5502542, 'biotype': 'lncRNA'}
CU633967.1 {'gene_id': 'ENSG00000274333', 'chromsome': 21, 'start': 5553637, 'end': 5614880, 'biotype': 'lncRNA'}
Y_RNA {'gene_id': 'ENSG00000276902', 'chromsome': 21, 'start': 6365955, 'end': 6366055, 'biotype': 'misc_RNA'}
FP236315.2 {'gene_id': 'ENSG00000279186', 'chromsome': 21, 'start': 5703182, 'end': 5705637, 'biotype': 'TEC'}
FP236315.3 {'gene_id': 'ENSG00000279784', 'chromsome': 21, 'start': 5705345, 'end': 5707160, 'biotype': 'lncRNA'}
FP236315.1 {'gene_id': 'ENSG00000279064', 'chromsome': 21, 'start': 5707004, 'end': 5709456, 'biotype': 'lncRNA'}
CU639417.3 {'gene_id': 'ENSG00000280013', 'chromsome': 21, 'start': 6008604, 'end': 6008810, 'b

# defining functions

In [45]:
def f_to_c(fahrenheit):
    celsius = (float(fahrenheit) - 32) * 5 / 9
    return round(celsius, 2)

In [47]:
ftemp = 110
ctemp = f_to_c(ftemp)
ctemp

43.33

In [48]:
print('C =', f_to_c(ftemp))

C = 43.33


In [49]:
def roman(string):
    '''
    Returns the integer value of a string in Roman numerals
    ''' 
    dictionary = {'M':1000, 'D':500, 'C':100, 'L':50, 'X':10, 'V':5, 'I':1}
    arabic = []
    # list that holds the Roman numerals as Arabic ciphers
    for letter in string.upper():
        if letter in dictionary:
            arabic.append(dictionary[letter])

    for i in range(len(arabic) - 1):
        if arabic[i] < arabic[i+1]:
            arabic[i] *= -1 # (5,10) -> (-5,10)
    print(arabic)
    return sum(arabic)

In [52]:
help(roman)

Help on function roman in module __main__:

roman(string)
    Returns the integer value of a string in Roman numerals



In [51]:
while (True):
    print(roman(input('Tell me a number [roman numerals]: ')))
    arabic = []

Tell me a number [roman numerals]: MCMXCIII
[1000, -100, 1000, -10, 100, 1, 1, 1]
1993
Tell me a number [roman numerals]: MCM
[1000, -100, 1000]
1900


KeyboardInterrupt: 

In [53]:
def seq_list_from_fastq_file(filename):
    '''
    Parse fastq file returning two lists, one of all sequences
    the other of the quality strings
    '''
    seq_list = []
    quality_list = []
    
    with open(filename) as FASTQ_INPUT:
        line_cnt = 0
        for line in FASTQ_INPUT:
            line_cnt += 1
            if line_cnt == 2:
                seq_list.append(line.rstrip('\n'))
            if line_cnt % 4 == 0:
                quality_list.append(line.rstrip('\n'))
                line_cnt = 0

    return seq_list, quality_list

In [54]:
# Use this function to return two lists:
#
# sequences, qualities = seq_list_from_fastq_file(fastq_filename)

In [55]:
DNA_codon_table = {
#                        Second Base
#        T             C             A             G
# T
    'TTT': 'Phe', 'TCT': 'Ser', 'TAT': 'Tyr', 'TGT': 'Cys',     # TxT
    'TTC': 'Phe', 'TCC': 'Ser', 'TAC': 'Tyr', 'TGC': 'Cys',     # TxC
    'TTA': 'Leu', 'TCA': 'Ser', 'TAA': '---', 'TGA': '---',     # TxA
    'TTG': 'Leu', 'TCG': 'Ser', 'TAG': '---', 'TGG': 'Trp',     # TxG
# C
    'CTT': 'Leu', 'CCT': 'Pro', 'CAT': 'His', 'CGT': 'Arg',     # CxT
    'CTC': 'Leu', 'CCC': 'Pro', 'CAC': 'His', 'CGC': 'Arg',     # CxC
    'CTA': 'Leu', 'CCA': 'Pro', 'CAA': 'Gln', 'CGA': 'Arg',     # CxA
    'CTG': 'Leu', 'CCG': 'Pro', 'CAG': 'Gln', 'CGG': 'Arg',     # CxG
# A
    'ATT': 'Ile', 'ACT': 'Thr', 'AAT': 'Asn', 'AGT': 'Ser',     # AxT
    'ATC': 'Ile', 'ACC': 'Thr', 'AAC': 'Asn', 'AGC': 'Ser',     # AxC
    'ATA': 'Ile', 'ACA': 'Thr', 'AAA': 'Lys', 'AGA': 'Arg',     # AxA
    'ATG': 'Met', 'ACG': 'Thr', 'AAG': 'Lys', 'AGG': 'Arg',     # AxG
# G
    'GTT': 'Val', 'GCT': 'Ala', 'GAT': 'Asp', 'GGT': 'Gly',     # GxT
    'GTC': 'Val', 'GCC': 'Ala', 'GAC': 'Asp', 'GGC': 'Gly',     # GxC
    'GTA': 'Val', 'GCA': 'Ala', 'GAA': 'Glu', 'GGA': 'Gly',     # GxA
    'GTG': 'Val', 'GCG': 'Ala', 'GAG': 'Glu', 'GGG': 'Gly'      # GxG
}

dna_seq = 'ATGATATGGAGGAGGTAGCCGCGCGCCATGCGCGCTATATTTTGGTAT'

In [56]:
def translate_DNA_codon(codon):
    return DNA_codon_table[codon]

In [59]:
def translate(seq):
    '''Return the animo acid sequence corresponding to the DNA
    sequence seq'''
    translation = ''
    for n in range(0, len(seq) - (len(seq) % 3), 3): # every third base
        translation += translate_DNA_codon(seq[n:n+3])
    return translation

In [60]:
translate(dna_seq)

'MetIleTrpArgArg---ProArgAlaMetArgAlaIlePheTrpTyr'

In [61]:
def translate_in_frame(seq, framenum):
    '''Return the translation of seq in framenum 1, 2, or 3'''
    return translate(seq[framenum-1:])

In [63]:
translate_in_frame(dna_seq, 2)

'---TyrGlyGlyGlySerArgAlaProCysAlaLeuTyrPheGly'

In [64]:
def print_translation_in_frame(seq, framenum, prefix):
    '''Print the translation of seq in framenum preceded by prefix'''
    print(prefix,
          framenum,
          ' ' * framenum,
          translate_in_frame(seq, framenum),
          sep='')

In [66]:
help(print_translation_in_frame)

Help on function print_translation_in_frame in module __main__:

print_translation_in_frame(seq, framenum, prefix)
    Print the translation of seq in framenum preceded by prefix



In [68]:
print_translation_in_frame(dna_seq, 2, 'Frame ')

Frame 2  ---TyrGlyGlyGlySerArgAlaProCysAlaLeuTyrPheGly


In [69]:
def print_translations(seq, prefix = ''):
    '''Print the translations of seq in all three reading frames,
    each preceded by prefix'''
    # print DNA sequence, indented to line up after prefix is added
    # to translation lines
    print('\n', ' ' * (len(prefix) + 2), seq, sep='')
    for framenum in range(1,4):
        print_translation_in_frame(seq, framenum, prefix)

In [71]:
print_translations(dna_seq)


  ATGATATGGAGGAGGTAGCCGCGCGCCATGCGCGCTATATTTTGGTAT
1 MetIleTrpArgArg---ProArgAlaMetArgAlaIlePheTrpTyr
2  ---TyrGlyGlyGlySerArgAlaProCysAlaLeuTyrPheGly
3   AspMetGluGluValAlaAlaArgHisAlaArgTyrIleLeuVal


In [72]:
def translate_with_open_reading_frames(seq, framenum):
    '''Return the translation of seq in framenum (1, 2, or 3), with
    ---'s when not within an open reading frame; assume the read is
    not in an open frame when at the beginning of seq'''
    open = False
    translation = ""
    seqlength = len(seq) - (framenum - 1)
    for n in range(framenum-1, seqlength - (seqlength % 3), 3):
        codon = translate_DNA_codon(seq[n:n+3])
        open = (open or codon == "Met") and not (codon == "---")
        translation += codon if open else "---"
    return translation

In [133]:
trans_dna = translate_with_open_reading_frames(dna_seq, 1)
trans_dna

'MetIleTrpArgArg------------MetArgAlaIlePheTrpTyr'

In [74]:
def print_translation_with_open_reading_frame(seq, framenum, prefix):
    print(prefix,
          framenum,
          ' ' * framenum,
          translate_with_open_reading_frames(seq, framenum),
          sep='')

In [84]:
print_translation_with_open_reading_frame(dna_seq, 1, 'Seq ')

Seq 1 MetIleTrpArgArg------------MetArgAlaIlePheTrpTyr


In [76]:
def print_translations_with_open_reading_frames(seq, prefix=''):
    print('\n', ' ' * (len(prefix) + 2), seq, sep='')
    for frame in range(1,4):
        print_translation_with_open_reading_frame(seq, frame, prefix)

In [86]:
print_translations_with_open_reading_frames(dna_seq)


  ATGATATGGAGGAGGTAGCCGCGCGCCATGCGCGCTATATTTTGGTAT
1 MetIleTrpArgArg------------MetArgAlaIlePheTrpTyr
2  ---------------------------------------------
3   ---MetGluGluValAlaAlaArgHisAlaArgTyrIleLeuVal


In [78]:
def print_translations_in_frames_in_both_directions(seq):
    print_translations(seq, '5\'->3\' ORF ')
    print_translations(seq[::-1], '3\'->5\' ORF ')

In [80]:
print_translations_in_frames_in_both_directions(dna_seq)


             ATGATATGGAGGAGGTAGCCGCGCGCCATGCGCGCTATATTTTGGTAT
5'->3' ORF 1 MetIleTrpArgArg---ProArgAlaMetArgAlaIlePheTrpTyr
5'->3' ORF 2  ---TyrGlyGlyGlySerArgAlaProCysAlaLeuTyrPheGly
5'->3' ORF 3   AspMetGluGluValAlaAlaArgHisAlaArgTyrIleLeuVal

             TATGGTTTTATATCGCGCGTACCGCGCGCCGATGGAGGAGGTATAGTA
3'->5' ORF 1 TyrGlyPheIleSerArgValProArgAlaAspGlyGlyGlyIleVal
3'->5' ORF 2  MetValLeuTyrArgAlaTyrArgAlaProMetGluGluVal---
3'->5' ORF 3   TrpPheTyrIleAlaArgThrAlaArgArgTrpArgArgTyrSer


In [81]:
def print_translations_with_open_reading_frames_in_both_directions(seq):
    print_translations_with_open_reading_frames(seq, '5\'->3\' ORF ')
    print_translations_with_open_reading_frames(seq[::-1], '3\'->5\' ORF ')

In [87]:
print_translations_with_open_reading_frames_in_both_directions(dna_seq)


             ATGATATGGAGGAGGTAGCCGCGCGCCATGCGCGCTATATTTTGGTAT
5'->3' ORF 1 MetIleTrpArgArg------------MetArgAlaIlePheTrpTyr
5'->3' ORF 2  ---------------------------------------------
5'->3' ORF 3   ---MetGluGluValAlaAlaArgHisAlaArgTyrIleLeuVal

             TATGGTTTTATATCGCGCGTACCGCGCGCCGATGGAGGAGGTATAGTA
3'->5' ORF 1 ------------------------------------------------
3'->5' ORF 2  MetValLeuTyrArgAlaTyrArgAlaProMetGluGluVal---
3'->5' ORF 3   ---------------------------------------------


# map

In [88]:
# starting list
mylist = [1.2, 3.2, 5.4, 2.1, 0.8]

In [91]:
# round all values in list to nearest int
print(list(map(round, mylist)))

[1, 3, 5, 2, 1]


In [93]:
map(round, mylist)

In [95]:
# define function to square values
def sqr(x):
    return round(pow(x,2),3)

In [96]:
# square each value of list
print(list(map(sqr, mylist)))

[1.44, 10.24, 29.16, 4.41, 0.64]


In [97]:
# mylist #2
mylist2 = ['71', '72', '73', '74']

In [100]:
# convert each value in list to binary (base 2)
print(list(map(chr, map(int, mylist2))))

['G', 'H', 'I', 'J']


# list comprehension

<b>[</b> expression <b>for</b> item <b>in</b> list <b>if</b> conditional <b>]</b>

<code>for item in list:
    if conditional:
        expression</code>

In [102]:
mylist = []

for x in 'spam':
    mylist.append(ord(x))
print(mylist)

[115, 112, 97, 109]


In [105]:
# convert a string into a list of ASCII values
[ord(x) for x in 'spam']

[115, 112, 97, 109]

In [106]:
# convert a string into a set of ASCII values
{ord(x) for x in 'spaam'}  # square to curly

{97, 109, 112, 115}

In [109]:
# convert a string into a dict of ASCII values
{x: ord(x) for x in 'spaam'}

{'s': 115, 'p': 112, 'a': 97, 'm': 109}

In [110]:
S = []
V = []
M = []
for x in range(10):
    S.append(x**2)
print('S =', S)

for i in range(13):
    V.append(2**i)
print('V =', V)

for x in S:
    if x % 2 == 0:
        M.append(x)
print('M =', M)

S = [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
V = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]
M = [0, 4, 16, 36, 64]


In [112]:
S = [x**2 for x in range(10)]
V = [2**x for x in range(13)]
M = [x for x in S if x % 2 == 0]
print('S =', S)
print('V =', V)
print('M =', M)

S = [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
V = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]
M = [0, 4, 16, 36, 64]


In [116]:
# start with a matrix, in this case represented as three lists
M = [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]]
col2 = []

In [117]:
# print out just the 2nd column
for row in M:
    col2.append(row[1])
print(col2)

[2, 5, 8]


In [119]:
[row[1] for row in M]

[2, 5, 8]

In [120]:
# Nested comprehensions
# tuples that are the cross products of two lists: [-1,0,1] and [-1,0,1]
mylist = []
for x in range(-1,2):
    for y in range(-1,2):
        if x == y:
            mylist.append((x,y))

print(mylist)

[(-1, -1), (0, 0), (1, 1)]


In [121]:
[(x, y) for x in range(-1,2) for y in range(-1,2) if x == y]

[(-1, -1), (0, 0), (1, 1)]

In [124]:
numbers = [1, 2, 3, 5, 7, 8, 11, 10, 11, 14]
doubled_odds = []

for n in numbers:
    if n % 2 == 1:
        doubled_odds.append(n * 2)
print(doubled_odds)

[2, 6, 10, 14, 22, 22]


In [125]:
doubled_odds = [n * 2 for n in numbers if n % 2 == 1]
print (doubled_odds)

[2, 6, 10, 14, 22, 22]


In [126]:
# output increasing amount of string
string = 'elephants'
[string[:c + 1] for c in range(len(string))]

['e',
 'el',
 'ele',
 'elep',
 'eleph',
 'elepha',
 'elephan',
 'elephant',
 'elephants']

In [127]:
# generate all k-mers of 23bp for a string of nucleotides
#                1         2         3         4
#      01234567890123456789012345678901234567890
dna = 'ACTGATCGATTACGTATAGTAGAATTCTATCATATATATGG'

kmers = []

for c in range(0, len(dna) - 22):
    kmers.append(dna[c:c+23])

kmers
#print(kmers)

['ACTGATCGATTACGTATAGTAGA',
 'CTGATCGATTACGTATAGTAGAA',
 'TGATCGATTACGTATAGTAGAAT',
 'GATCGATTACGTATAGTAGAATT',
 'ATCGATTACGTATAGTAGAATTC',
 'TCGATTACGTATAGTAGAATTCT',
 'CGATTACGTATAGTAGAATTCTA',
 'GATTACGTATAGTAGAATTCTAT',
 'ATTACGTATAGTAGAATTCTATC',
 'TTACGTATAGTAGAATTCTATCA',
 'TACGTATAGTAGAATTCTATCAT',
 'ACGTATAGTAGAATTCTATCATA',
 'CGTATAGTAGAATTCTATCATAT',
 'GTATAGTAGAATTCTATCATATA',
 'TATAGTAGAATTCTATCATATAT',
 'ATAGTAGAATTCTATCATATATA',
 'TAGTAGAATTCTATCATATATAT',
 'AGTAGAATTCTATCATATATATG',
 'GTAGAATTCTATCATATATATGG']

In [129]:
# generate all 23-mers of dna
[dna[c:c+23] for c in range(0, len(dna) - 22)]

['ACTGATCGATTACGTATAGTAGA',
 'CTGATCGATTACGTATAGTAGAA',
 'TGATCGATTACGTATAGTAGAAT',
 'GATCGATTACGTATAGTAGAATT',
 'ATCGATTACGTATAGTAGAATTC',
 'TCGATTACGTATAGTAGAATTCT',
 'CGATTACGTATAGTAGAATTCTA',
 'GATTACGTATAGTAGAATTCTAT',
 'ATTACGTATAGTAGAATTCTATC',
 'TTACGTATAGTAGAATTCTATCA',
 'TACGTATAGTAGAATTCTATCAT',
 'ACGTATAGTAGAATTCTATCATA',
 'CGTATAGTAGAATTCTATCATAT',
 'GTATAGTAGAATTCTATCATATA',
 'TATAGTAGAATTCTATCATATAT',
 'ATAGTAGAATTCTATCATATATA',
 'TAGTAGAATTCTATCATATATAT',
 'AGTAGAATTCTATCATATATATG',
 'GTAGAATTCTATCATATATATGG']

# zip()

In [130]:
# reverse key and value in mydict
mydict = {'A':12, 'B':25, 'C':23, 'D':14}
reversed = dict(zip(mydict.values(), mydict))
print(reversed)

{12: 'A', 25: 'B', 23: 'C', 14: 'D'}


In [131]:
# same as above
reversed = dict(zip(mydict.values(), mydict.keys()))
print(reversed)

{12: 'A', 25: 'B', 23: 'C', 14: 'D'}
